In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv
from arviz import hdi
sns.set_theme(style='white', context='notebook', font_scale=1.33)

## Section 1: 1PL-G Models

### 1.1 Model comparison

In [2]:
for i, m in enumerate(['1plg_m1','1plg_m2','1plg_m3','1plg_m4']):
    
    ## Load ppc.
    ppc = read_csv(os.path.join('stan_results', f'{m}_ppc.csv'))
        
    ## Model comparison.
    loco = ppc.groupby('subject').loco.mean().sum()
    pwaic_c = ppc.groupby('subject').pwaic_c.mean().sum()
    
    print('%s: %0.2f | %0.2f' %(m, loco, pwaic_c))

1plg_m1: -14516.85 | 93.29
1plg_m2: -13804.74 | 159.38
1plg_m3: -13481.12 | 164.26
1plg_m4: -13293.16 | 339.87


In [3]:
from itertools import combinations

for a, b in list(combinations(['1plg_m1','1plg_m2','1plg_m3','1plg_m4'], 2)):
    
    ## Load data.
    df1 = read_csv(os.path.join('stan_results', f'{a}_ppc.csv'))
    df2 = read_csv(os.path.join('stan_results', f'{b}_ppc.csv'))
    
    arr = df2.groupby('subject').loco.mean() - df1.groupby('subject').loco.mean()
    
    ## Compute stats.
    N = df1.subject.nunique()
    mu = np.sum(arr)
    se = np.std(arr) * np.sqrt(N)
    
    print(f'{b} - {a}: %0.2f (%0.2f)' %(mu, se))

1plg_m2 - 1plg_m1: 712.11 (34.61)
1plg_m3 - 1plg_m1: 1035.73 (41.95)
1plg_m4 - 1plg_m1: 1223.69 (44.94)
1plg_m3 - 1plg_m2: 323.62 (23.36)
1plg_m4 - 1plg_m2: 511.58 (28.77)
1plg_m4 - 1plg_m3: 187.96 (17.25)


### 1.2 Parameter distributions

In [4]:
## Load samples.
samples = read_csv(os.path.join('stan_results', '1plg_m4.tsv.gz'), sep='\t', compression='gzip')

## Extract parameters.
beta_mu   = samples.filter(regex='beta_mu\[').values
beta_1_pr = samples.filter(regex='beta_1_pr\[').values
beta      = samples.filter(regex='beta\[').values
sigma     = samples.filter(regex='sigma\[').values

## Item difficulty (mean).
mu = beta.mean()
lb, ub = hdi(beta.mean(axis=1), hdi_prob=0.95)
print('beta (mean):\t%0.3f [%0.3f, %0.3f]' %(mu, lb, ub))

## Item difficulty (variance).
mu = beta.std(axis=1).mean()
lb, ub = hdi(beta.std(axis=1), hdi_prob=0.95)
print('beta (var):\t%0.3f [%0.3f, %0.3f]' %(mu, lb, ub))

beta (mean):	0.177 [0.118, 0.238]
beta (var):	1.431 [1.370, 1.492]


### 1.3 Interpreting parameters

In [5]:
## Define useful functions.
def inv_logit(x):
    return 1. / (1 + np.exp(-x))

## Load design.
X2 = read_csv(os.path.join('designs', 'X2.csv'))

## Iterate over fixed effects.
for i, col in enumerate(['n_features', 'n_rules', 'distractor', 'rt']):
    
    ## Compute regressor SD.
    sd = X2[col].std()
    
    ## Compute change in response.
    delta = inv_logit(beta_mu[:,0] + 0.5 * (beta_mu[:,i+1] / sd))  -\
            inv_logit(beta_mu[:,0] - 0.5 * (beta_mu[:,i+1] / sd))
    
    ## Compute summary stats.
    mu = delta.mean()
    lb, ub = hdi(delta, hdi_prob=0.95)
    
    ## Print.
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(col[:5],mu,lb,ub))

n_fea:	0.132 [0.087, 0.174]
n_rul:	0.112 [0.080, 0.144]
distr:	0.252 [0.218, 0.287]
rt:	0.122 [0.093, 0.151]


### 1.4 Variance explained

In [6]:
## Load and prepare design matrices.
zscore = lambda x: (x - np.nanmean(x)) / np.nanstd(x)
X2 = read_csv(os.path.join('designs', 'X2.csv')).apply(zscore, 0).fillna(1).values
X3 = read_csv(os.path.join('designs', 'X3.csv')).values.astype(float)

## Compute total variance.
total_var = np.var(beta, axis=1).mean()

## Compute explained variance (level-1 fixed effects).
var = np.var(beta - beta_mu[:,[1,2]] @ X2.T[[1,2]], axis=1).mean()
print('level-1   fixed:\t %0.3f' %(1 - var / total_var))

## Compute explained variance (item fixed effects).
var = np.var(beta - beta_mu[:,[1,2,3]] @ X2.T[[1,2,3]], axis=1).mean()
print('all item  fixed:\t %0.3f' %(1 - var / total_var))

## Compute explained variance (level-1,2 fixed effects).
var = np.var(beta - beta_mu @ X2.T, axis=1).mean()
print('level-1,2 fixed:\t %0.3f' %(1 - var / total_var))

## Compute variance explained (all effects).
mu = beta_mu @ X2.T + beta_1_pr @ X3.T * np.atleast_2d(sigma[:,0]).T
var = np.var(beta - mu, axis=1).mean()
print('all effects:    \t %0.3f' %(1 - var / total_var))

level-1   fixed:	 0.350
all item  fixed:	 0.482
level-1,2 fixed:	 0.656
all effects:    	 0.841


## Section 2: 3PL Models

### 2.1 Model comparison

In [7]:
for i, m in enumerate(['3pl_m1','3pl_m2','3pl_m3']):
    
    ## Load ppc.
    ppc = read_csv(os.path.join('stan_results', f'{m}_ppc.csv'))
        
    ## Model comparison.
    loco = ppc.groupby('subject').loco.mean().sum()
    pwaic_c = ppc.groupby('subject').pwaic_c.mean().sum()
    
    print('%s: %0.2f | %0.2f' %(m, loco, pwaic_c))

3pl_m1: -13209.38 | 384.90
3pl_m2: -13207.25 | 408.13
3pl_m3: -13208.69 | 426.78


In [8]:
from itertools import combinations

for a, b in list(combinations(['3pl_m1','3pl_m2','3pl_m3'], 2)):
    
    ## Load data.
    df1 = read_csv(os.path.join('stan_results', f'{a}_ppc.csv'))
    df2 = read_csv(os.path.join('stan_results', f'{b}_ppc.csv'))
    
    arr = df2.groupby('subject').loco.mean() - df1.groupby('subject').loco.mean()
    
    ## Compute stats.
    N = df1.subject.nunique()
    mu = np.sum(arr)
    se = np.std(arr) * np.sqrt(N)
    
    print(f'{b} - {a}: %0.2f (%0.2f)' %(mu, se))

3pl_m2 - 3pl_m1: 2.13 (2.04)
3pl_m3 - 3pl_m1: 0.69 (2.20)
3pl_m3 - 3pl_m2: -1.44 (0.85)


### 2.2 Parameter distributions

In [9]:
## Load samples.
samples = read_csv(os.path.join('stan_results', '3pl_m2.tsv.gz'), sep='\t', compression='gzip')

## Extract parameters.
beta_mu   = samples.filter(regex='beta_mu\[').values
beta_1_pr = samples.filter(regex='beta_1_pr\[').values
beta      = samples.filter(regex='beta\[').values
alpha_mu   = samples.filter(regex='alpha_mu\[').values
alpha      = samples.filter(regex='alpha\[').values
sigma     = samples.filter(regex='sigma\[').values
c = -0.625

## Item difficulty (mean).
mu = beta.mean()
lb, ub = hdi(beta.mean(axis=1), hdi_prob=0.95)
print('beta (mean):\t%0.3f [%0.3f, %0.3f]' %(mu, lb, ub))

## Item difficulty (variance).
mu = beta.std(axis=1).mean()
lb, ub = hdi(beta.std(axis=1), hdi_prob=0.95)
print('beta (var):\t%0.3f [%0.3f, %0.3f]' %(mu, lb, ub))

## Item discrimination (mean).
mu = alpha.mean()
lb, ub = hdi(alpha.mean(axis=1), hdi_prob=0.95)
print('alpha (mean):\t%0.3f [%0.3f, %0.3f]' %(mu, lb, ub))

## Item difficulty (variance).
mu = alpha.std(axis=1).mean()
lb, ub = hdi(beta.std(axis=1), hdi_prob=0.95)
print('alpha (var):\t%0.3f [%0.3f, %0.3f]' %(mu, lb, ub))

beta (mean):	0.221 [0.142, 0.302]
beta (var):	1.554 [1.475, 1.629]
alpha (mean):	1.298 [1.212, 1.385]
alpha (var):	0.221 [1.475, 1.629]


### 2.3 Interpreting parameters

#### Item difficulty

In [10]:
## Define useful functions.
def inv_logit(x):
    return 1. / (1 + np.exp(-x))

## Load design.
X2 = read_csv(os.path.join('designs', 'X2.csv'))

## Iterate over fixed effects.
for i, col in enumerate(['n_features', 'n_rules', 'distractor', 'rt']):
    
    ## Compute regressor SD.
    sd = X2[col].std()
    
    ## Compute change in response.
    delta = inv_logit(beta_mu[:,0] + 0.5 * (beta_mu[:,i+1] / sd))  -\
            inv_logit(beta_mu[:,0] - 0.5 * (beta_mu[:,i+1] / sd))
    
    ## Compute summary stats.
    mu = delta.mean()
    lb, ub = hdi(delta, hdi_prob=0.95)
    
    ## Print.
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(col[:5],mu,lb,ub))

n_fea:	0.142 [0.096, 0.188]
n_rul:	0.126 [0.093, 0.162]
distr:	0.266 [0.229, 0.305]
rt:	0.132 [0.100, 0.163]


#### Item discrimination

In [11]:
## Iterate over fixed effects.
for i, col in enumerate(['n_features', 'n_rules', 'distractor', 'rt']):
    
    ## Compute summary stats.
    mu = alpha_mu[:,i+1].mean()
    lb, ub = hdi(alpha_mu[:,i+1], hdi_prob=0.95)
    
    ## Print.
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(col[:5],mu,lb,ub))

n_fea:	-0.037 [-0.090, 0.017]
n_rul:	0.067 [0.011, 0.123]
distr:	-0.036 [-0.082, 0.006]
rt:	-0.025 [-0.075, 0.024]


### 2.4 Variance explained

#### Item difficulty

In [12]:
## Load and prepare design matrices.
zscore = lambda x: (x - np.nanmean(x)) / np.nanstd(x)
X2 = read_csv(os.path.join('designs', 'X2.csv')).apply(zscore, 0).fillna(1).values
X3 = read_csv(os.path.join('designs', 'X3.csv')).values.astype(float)

## Compute total variance.
total_var = np.var(beta, axis=1).mean()

## Compute explained variance (level-1 fixed effects).
var = np.var(beta - beta_mu[:,[1,2]] @ X2.T[[1,2]], axis=1).mean()
print('level-1   fixed:\t %0.3f' %(1 - var / total_var))

## Compute explained variance (item fixed effects).
var = np.var(beta - beta_mu[:,[1,2,3]] @ X2.T[[1,2,3]], axis=1).mean()
print('all item  fixed:\t %0.3f' %(1 - var / total_var))

## Compute explained variance (level-1,2 fixed effects).
var = np.var(beta - beta_mu @ X2.T, axis=1).mean()
print('level-1,2 fixed:\t %0.3f' %(1 - var / total_var))

## Compute variance explained (all effects).
mu = beta_mu @ X2.T + beta_1_pr @ X3.T * np.atleast_2d(sigma[:,0]).T
var = np.var(beta - mu, axis=1).mean()
print('all effects:    \t %0.3f' %(1 - var / total_var))

level-1   fixed:	 0.364
all item  fixed:	 0.491
level-1,2 fixed:	 0.662
all effects:    	 0.841


#### Item Discrimination

In [13]:
from scipy.stats import norm

## Load and prepare design matrices.
zscore = lambda x: (x - np.nanmean(x)) / np.nanstd(x)
X2 = read_csv(os.path.join('designs', 'X2.csv')).apply(zscore, 0).fillna(1).values

## Compute total variance.
total_var = np.var(alpha, axis=1).mean()

## Variance explained (level-1).
mu = norm.cdf(c + alpha_mu @ X2.T) * 5
var = np.var(alpha - mu, axis=1).mean()
print('level-1,2 fixed:\t %0.3f' %(1 - var / total_var))

level-1,2 fixed:	 0.449


## Section 3: Explanatory IRT

In [14]:
## Extract parameters.
theta = samples.filter(regex='theta\[').values
rho   = samples.filter(regex='rho\[').values

### 3.1 Interpreting parameters

In [15]:
## Load and prepare design matrices.
X1 = read_csv(os.path.join('designs', 'X1.csv'))

for i, col in enumerate(X1.columns):
    
    ## Compute summary stats.
    mu = rho[:,i].mean()
    lb, ub = hdi(rho[:,i], hdi_prob=0.95)
    
    ## Print.
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(col[:5],mu,lb,ub))

age:	-0.299 [-0.353, -0.246]
gende:	0.128 [0.078, 0.182]
rt_0:	0.427 [0.374, 0.478]
rt_1:	0.368 [0.313, 0.424]


#### 3.2 Variance explained

In [16]:
## Load and prepare design matrices.
zscore = lambda x: (x - np.nanmean(x)) / np.nanstd(x)
X1 = read_csv(os.path.join('designs', 'X1.csv')).apply(zscore, 0).fillna(1).values

## Compute total variance.
total_var = np.var(theta, axis=1).mean()

## Variance explained (level-1).
var = np.var(theta - rho @ X1.T, axis=1).mean()
print('all effects: \t %0.3f' %(1 - var / total_var))

all effects: 	 0.502


## Section 4: Model Diagnostics

In [ ]:
df = read_csv('stan_results/3pl_m2_ppc.csv')
sx = read_csv('stan_results/3pl_m2_ifdm.csv', index_col=0)
gb = df.groupby('item_id').agg({'accuracy':'mean', 'y_hat':'mean'})
gb = gb.merge(sx, left_index=True, right_index=True)
gb['thresh'] = np.where(np.logical_or(gb.pval < 0.05, gb.pval > 0.95), 1, 0)
ax = sns.scatterplot(x='accuracy', y='y_hat', hue='thresh', data=gb)
ax.plot([0,1],[0,1])